### Use Machine Learning to predict the probability of needing Post-Mastectomy Radiotherapy
Author: Yifu (Charles) Chen

A notebook in Python Language

In [8]:
### Import modules ###
import os
import glob
import pandas as pd
import numpy as np
from typing import Dict, List
import re
# Machine Learning Modules for Random Forest, Logistic Regression, and XGBoost
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
# Metrics
from sklearn import metrics

### Utils

In [9]:
# Setup no print limits and etc.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', False)

class bcolors:
    # Helper class to print in terminal with colors
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    NORMAL = '\033[0m'

In [ ]:
from collections import OrderedDict

# Directory of DataFrames
dir_to_df = "/Users/yifu/PycharmProjects/Radiotherapy-Prediction/data/experiments/May26 Datasets for Experiments Table (enhanced expert cols)"

# Path to log file
log_path = os.path.join(dir_to_df, "log.txt")

# Create a new log file from scratch, regardless if one exists
with open(log_path, "w") as log_file:
    log_file.write("")

def print_and_log(*args, **kwargs):
    # Helper function to print and log
    # Can write DataFrame to log file too
    print(*args, **kwargs)
    with open(log_path, "a") as log_file:
        if type(args[0]) == pd.DataFrame:
            log_file.write(args[0].to_string() + "\n")
        else:
            log_file.write("|| ".join([str(a) for a in args]) + "\n")

    
# Assume all files in directory are CSV DataFrame files
paths_to_df = glob.glob(os.path.join(dir_to_df, "*.csv"))

# Read each DataFrame file
dfs = OrderedDict()
for i, path in enumerate(paths_to_df):
    df_name = path.replace(dir_to_df, "")
    df = pd.read_csv(path)
    if i == len(paths_to_df) - 1:
        print("Kept the largest DF")
        df_all = df.copy()
        # dfs["df_all"] = df
    # Drop "PRE_record_id" column, if it exists
    if "PRE_record_id" in df.columns:
        df = df.drop(columns=["PRE_record_id"], axis=1)
    dfs[df_name] = df
# Sorted the dict by path name
dfs = OrderedDict(sorted(dfs.items()))

for k in dfs.keys():
    print_and_log(k)
    



### Pre-Process Datasets

In [ ]:
# Standardize the data by removing the mean and scaling to unit variance
# The target variable is named "POS_did_the_patient_receive_pm"

dfs_std = OrderedDict()

for df_name, df in dfs.items():
    # Standardize all columns except the target column "POS_did_the_patient_receive_pm"
    df_std = df.copy()
    df_std.drop(["POS_did_the_patient_receive_pm"], axis=1, inplace=True)
    df_std = (df_std - df_std.mean()) / df_std.std()
    df_std = df_std.copy()
    # Concatenate the target column back
    df_std["POS_did_the_patient_receive_pm"] = df["POS_did_the_patient_receive_pm"]
    # Convert the target column to binary
    df_std["POS_did_the_patient_receive_pm"] = df_std["POS_did_the_patient_receive_pm"] > 0
    dfs_std[df_name] = df_std

len(dfs_std)

13

### Build Machine Learning Classifiers

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:


# Initialize a dictionary for holding the models for each DataFrame
# Schema: {"DataFrame Name": {"Model Name": Model Object, ...}, ...}
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import itertools

all_models = defaultdict(dict)

# The print_df contains the dataframe, model, and the average CV accuracy, AUC, and the top 6 features and their scores
# print_df_columns = ["Dataset", "Model", "Avg CV Accuracy", "Avg CV AUC", "Avg CV F1", "Feature_1", "Score_1", "Feature_2", "Score_2", 
#                     "Feature_3", "Score_3", "Feature_4", "Score_4", "Feature_5", "Score_5", "Feature_6", "Score_6"]

print_df_columns = ["Dataset", "Model"] + list(itertools.chain(*[["Avg " + m, "SE " + m] for m in ["Accuracy", "AUC", "F1"]]))

def build_classifiers(df: pd.DataFrame, df_name: str, num_folds: int=15) -> Dict[str, object]:
    """
    For each DataFrame, build the following classifiers:
        Logistic Regression
        Random Forest
        XGBoost
    Args:
        df (pd.DataFrame): the DataFrame to build the classifiers for
        df_name (str): the name of the DataFrame
    Returns:
        None
    """
    # Impute missing values with KNN
    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=5)
    # Drop columns that are all NaN, also drop the target column
    df_drop_na = df.dropna(axis=1, how="all")
    df_drop_na = df_drop_na.drop(columns=["POS_did_the_patient_receive_pm"])
    # Impute the missing values
    df_imp = pd.DataFrame(imputer.fit_transform(df_drop_na), columns=df_drop_na.columns)
    # Put the target back
    df_imp["POS_did_the_patient_receive_pm"] = df["POS_did_the_patient_receive_pm"]
    df = df_imp.copy()

    assert df.isnull().sum().sum() == 0

    # Initialize a dictionary for holding the models for each DataFrame
    # Schema: {"Fold 1": {"Model Name": Model Object, ...}, ...}
    models = defaultdict(dict)
    performance = defaultdict(dict)
    # Train each model using 5-fold cross-validation, repeated 10 times to get error bars
    # for i in range(num_cvs):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=0)
    for fold, (train_index, test_index) in enumerate(kf.split(df)):
        fold = fold + 1
        print("Fold {}".format(fold), end="... ")
        # Split the DataFrame into training and validation sets
        df_train, df_val = df.iloc[train_index], df.iloc[test_index]
        # Build the classifiers
        models[fold] = {"Logistic Regression": LogisticRegression(),
                        "Random Forest": RandomForestClassifier(),
                        "XGBoost": XGBClassifier()
                        }
        for model_name, model in models[fold].items():
            # Train each model on the target column POS_did_the_patient_receive_pm
            model.fit(
                df_train.drop(["POS_did_the_patient_receive_pm"], axis=1),
                df_train["POS_did_the_patient_receive_pm"]
                )
            # Predict the validation set
            y_pred = model.predict(
                df_val.drop(["POS_did_the_patient_receive_pm"], axis=1)
            )
            # Calculate the accuracy of the model
            accuracy = np.mean(y_pred == df_val["POS_did_the_patient_receive_pm"])
            # Calcuilate the F1
            F1 = f1_score(y_true=df_val["POS_did_the_patient_receive_pm"], y_pred=y_pred)
            # Predict the probability of the validation set
            y_pred_proba = model.predict_proba(df_val.drop(["POS_did_the_patient_receive_pm"], axis=1))[:, 1]
            # Calculate the AUC of the model
            auc = metrics.roc_auc_score(df_val["POS_did_the_patient_receive_pm"], y_pred_proba)
            if model_name != "Logistic Regression":
                # Get the features for the model
                feature_importances = pd.Series(model.feature_importances_, index=df_train.drop(["POS_did_the_patient_receive_pm"], axis=1).columns)
            else:
                # The features for the Logistic Regression model are the coefficients
                # Take absolute value of the coefficients
                feature_importances = pd.Series(np.abs(model.coef_[0]), index=df_train.drop(["POS_did_the_patient_receive_pm"], axis=1).columns)
            # Store the accuracy, AUC, and features for the model
            performance[model_name][fold] = {"accuracy": accuracy, "auc": auc, "F1": F1, "top_features": feature_importances}
    
    # Consolidate the top features for each model
    # Schema: {"Model Name": {"Feature_1": Score_1, ...}, ...}
    feature_scores = defaultdict(lambda: defaultdict(float))
    for model_name, fold in performance.items():
        for fold_num, fold_performance in fold.items():
            # Extract feature name and importance score
            for feature_name, score in fold_performance["top_features"].items():
                feature_scores[model_name][feature_name] += score
        # Divide the feature importance scores by the number of folds
        for feature_name, score in feature_scores[model_name].items():
            feature_scores[model_name][feature_name] /= len(fold)
            # Round to 2 decimal places
            feature_scores[model_name][feature_name] = round(feature_scores[model_name][feature_name], 4)
    
    # Print the performance of each model, including averaged across folds
    # DataFrame for printing results in a nice format
    # Columns: Dataset, Model, Avg CV Accuracy, SE Accuracy, Avg CV AUC, SE AUC, Avg CV F1, SE F1
    print_df = pd.DataFrame(columns=print_df_columns)
    for model_name, fold in performance.items():
        accuracy = round(np.mean([fold[fold_num]["accuracy"] for fold_num in fold.keys()]), 4)
        auc = round(np.mean([fold[fold_num]["auc"] for fold_num in fold.keys()]), 4)
        F1 = round(np.mean([fold[fold_num]["F1"] for fold_num in fold.keys()]), 4)
        accuracy_se = round(np.std([fold[fold_num]["accuracy"] for fold_num in fold.keys()]) / np.sqrt(len(fold)), 4)
        auc_se = round(np.std([fold[fold_num]["auc"] for fold_num in fold.keys()]) / np.sqrt(len(fold)), 4)
        F1_se = round(np.std([fold[fold_num]["F1"] for fold_num in fold.keys()]) / np.sqrt(len(fold)), 4)
        # Convert metric and metric_se from float to string format
        # accuracy_str = str(accuracy) + " (" + str(accuracy_se) + ")"
        # auc_str = str(auc) + " (" + str(auc_se) + ")"
        # F1_str = str(F1) + " (" + str(F1_se) + ")"
        # curr_df = pd.DataFrame(
        #             [
        #                 [df_name, model_name, accuracy_str, auc_str, F1_str] +
        #                 [feat_1, score_1, feat_2, score_2, feat_3, score_3, feat_4, score_4, feat_5, score_5, feat_6, score_6]
        #             ],
        #             columns=print_df_columns
        #         )
        curr_df = pd.DataFrame(
                    [
                        [df_name, model_name, accuracy, accuracy_se, auc, auc_se, F1, F1_se]
                    ],
                    columns=print_df_columns
                )
        print_df = pd.concat(
            [
                print_df,
                curr_df
            ],
            axis=0
        )
    # Print the DataFrame with nice formatting
    # print_and_log(print_df)

    # print_and_log("="*150)
    return print_df, feature_scores


# Build the classifiers for each DataFrame
print_df = pd.DataFrame(columns=print_df_columns)
# Create a dictionary for holding the DataFrames for plotting
plot_dict = OrderedDict()
# Create a dictionary for holding all the feature scores across datasets and models
# Schema: {"Dataset1": {"Model1": {"Feature_1": Score_1, ...}, ...}, ...}
all_top_features = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
for df_name, df in dfs_std.items():
    print_and_log(f"Processing {df_name}...")
    print_df, feature_scores = build_classifiers(df, df_name)
    # Transpose the DataFrame before printing it
    print_and_log(print_df.T)
    model_names = feature_scores.keys()
    for model in model_names:
        print_and_log("\n" + "-" * 10 + f"The most important features for {model}:" + "-" * 10)
        top_features = sorted(feature_scores[model].items(), key=lambda x: x[1], reverse=True)[:]
        for i, (feat, score) in enumerate(top_features):
            all_top_features[df_name][model][feat] = score
            # If the feature is in the top 1/3, print it
            if i < len(top_features) / 3:
                print_and_log(f"{feat}: {score}", end="\t")
        print_and_log("")
    print_and_log("="*150)
    # Add the DataFrame to the dictionary for plotting later
    plot_dict[df_name] = print_df

Processing /0_PRE-alan-heuristic7cols.csv...
Fold 1... Fold 2... Fold 3... Fold 4... Fold 5... Fold 6... Fold 7... Fold 8... Fold 9... 

KeyboardInterrupt: 

In [ ]:
# Use different proportion subsets of DataFrame and determine the performance
# Sort df_all by "PRE_record_id"
# df_all = df_all.sort_values(by="PRE_record_id")
# # Drop "PRE_record_id" column
# df_all = df_all.drop(["PRE_record_id"], axis=1)
# for percentile in range(10, 100, 10):
#     percentage = percentile/100
#     # Take the first percentage of the DataFrame
#     df_subset = df_all.iloc[:int(len(df_all)*percentage)]
#     # Train the classifiers on the subset
#     print_df = build_classifiers(df_subset, f"{percentage}% of DataFrame")

### Plotting

In [ ]:
print_df

,Dataset,Model,Avg Accuracy,SE Accuracy,Avg AUC,SE AUC,Avg F1,SE F1
0,/C_POS-1.0spars-expert-imputed128cols.csv,Logistic Regression,0.794,0.0185,0.8274,0.0192,0.7403,0.0241
0,/C_POS-1.0spars-expert-imputed128cols.csv,Random Forest,0.8163,0.0139,0.8889,0.0142,0.7551,0.0201
0,/C_POS-1.0spars-expert-imputed128cols.csv,XGBoost,0.8066,0.015,0.8839,0.0136,0.7498,0.0223


In [ ]:
# Import plotting libraries for machine learning experiments
import matplotlib.pyplot as plt
import seaborn as sns

plot_dir = os.path.join(dir_to_df, "plots")
# Create the directory if it doesn't exist
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

# Retrive the saved DataFrames, and plot the results
# Plot the results for each DataFrame on a single plot
# Reset plot parameters
plt.rcParams.update(plt.rcParamsDefault)
for metric_name in ["Accuracy", "AUC", "F1"]:
    y_data = []
    errors = []
    x_data = []
    # Retrieve the performance for each model
    for df_name, print_df in plot_dict.items():
        # Iterate through the models of the DataFrame
        for i, row in print_df.iterrows():
            # If the model is the metric, add it to the list
            model_name = row["Model"]
            # Use regular expression to retrieve the metric and its standard error
            metric = row["Avg " + metric_name]
            error = row["SE " + metric_name]
            df_name = row["Dataset"]
            df_name = df_name.replace("/", "").replace(".csv", "")
            x_data.append(metric)
            y_data.append(df_name + " - " + model_name)
            errors.append(error)
    # Create the plot
    # The plot has spaces between the bars
    plt.figure(figsize=(30, 16))
    # Plot the gridlines
    sns.set_style("whitegrid")
    # The color of the bar should alternate between 3 colors
    colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]
    # Make bars 
    plt.barh(y_data, x_data, xerr=errors, color=colors, align="center")
    # plt.barh(y_data, x_data, align="center", alpha=0.8)
    plt.yticks(fontsize=18)
    plt.xticks(fontsize=18)
    plt.xlabel(metric_name, fontsize=20)
    plt.ylabel("Model")
    plt.title(f"{metric_name} by Model", fontsize=20)
    plt.tight_layout()
    # Save the plot
    plt.savefig(os.path.join(plot_dir, f"{metric_name}.png"))
    plt.close()



In [ ]:
# for each dataset in all_top_features, build a DataFrame with the top features
# The features for each model is ranked by their score
# The most important features shared by all models are sorted to the top
# Columns: Features | Logistic Regression | Random Forest| XGBoost
# Row 1:    feature_1 | feature_1_rank_LR | feature_1_rank_RF | feature_1_rank_XGB
# Row 2:    feature_2 | feature_2_rank_LR | feature_2_rank_RF | feature_2_rank_XGB
 
# Schema: {"Dataset1": {"Model1": {"Feature_1": Score_1, ...}, ...}, ...}

# We currently only consider the Random Forest model
for dataset, model_dict in all_top_features.items():
    print_and_log("-"*50)
    print_and_log(f"Processing {dataset}...")
    LR_features = model_dict["Logistic Regression"]
    RF_features = model_dict["Random Forest"]
    XGB_features = model_dict["XGBoost"]
    # Sort the features by their scores
    sorted_LR_features = sorted(LR_features.items(), key=lambda x: x[1], reverse=True)
    sorted_RF_features = sorted(RF_features.items(), key=lambda x: x[1], reverse=True)
    sorted_XGB_features = sorted(XGB_features.items(), key=lambda x: x[1], reverse=True)
    # Convert each score into a rank
    rank_LR_features = {feat: i+1 for i, (feat, score) in enumerate(sorted_LR_features)}
    rank_RF_features = {feat: i+1 for i, (feat, score) in enumerate(sorted_RF_features)}
    rank_XGB_features = {feat: i+1 for i, (feat, score) in enumerate(sorted_XGB_features)}
    # Print the number of features for each model
    print_and_log(f"{dataset}: {len(sorted_LR_features)} features for Logistic Regression, {len(sorted_RF_features)} features for Random Forest, and {len(sorted_XGB_features)} features for XGBoost")
    # Assert all the features are present in the sorted features dictionary
    assert set(rank_LR_features.keys()) == set(rank_RF_features.keys()) == set(rank_XGB_features.keys())
    # Create a DataFrame with all the features
    df = pd.DataFrame({"Features": list(rank_LR_features.keys())})
    # Add the ranks of LR to the DataFrame
    df["LR_rank"] = df["Features"].map(rank_LR_features)
    # Add the ranks of RF to the DataFrame
    df["RF_rank"] = df["Features"].map(rank_RF_features)
    # Add the ranks of XGB to the DataFrame
    df["XGB_rank"] = df["Features"].map(rank_XGB_features)
    # Sort the DataFrame by the ranks of the features
    df = df.sort_values(by=["LR_rank", "RF_rank", "XGB_rank"])
    # Keep only the top 1/3 of features (the most important features)
    df = df.iloc[:int(len(df)*1/3)]
    
    # Print the DataFrame
    print(df)



--------------------------------------------------
Processing /0_PRE-alan-heuristic7cols.csv...
/0_PRE-alan-heuristic7cols.csv: 2 features for Logistic Regression, 2 features for Random Forest, and 2 features for XGBoost


AssertionError: 

In [ ]:
# # Print the Feature Importance
# # Format:
# # DataFrame | Model | Feature_1 | Score_1 | Feature_2 | Score_2 | Feature_3 | Score_3 | Feature_4 | Score_4 | Feature_5 | Score_5 | Feature_6 | Score_6
# for df_name, print_df in plot_dict.items():
#     df_name = df_name.replace("/", "").replace(".csv", "")
#     print_and_log(df_name)
#     print()
#     for i, row in print_df.iterrows():
#         model_name = row["Model"]
#         if model_name == "Logistic Regression":
#             continue
#         print_and_log("Model:", model_name)
#         top_features = [row["Feature_1"], row["Feature_2"], row["Feature_3"], row["Feature_4"], row["Feature_5"], row["Feature_6"]]
#         feature_scores = [row["Score_1"], row["Score_2"], row["Score_3"], row["Score_4"], row["Score_5"], row["Score_6"]]
#         print_and_log(f"Top Features: {top_features}")
#         print_and_log(f"Feature Scores: {feature_scores}")
#         print()
#     print_and_log("-"* 100)

0_PRE-alan-heuristic7cols

Model: Random Forest


KeyError: 'Feature_1'

In [ ]:
# # Plot the most important features and scores for each model
# # Create subplots for each DataFrame
# for df_name, print_df in plot_dict.items():
#     df_name = df_name.replace("/", "").replace(".csv", "")
#     # Create a plot for the top 6 important features of each model
#     # Y-Axis: Model + Feature Name
#     # X-Axis: Feature importance score
#     # Create subplots 
#     for i, row in print_df.iterrows():
#         top_features = [row["Feature_1"], row["Feature_2"], row["Feature_3"], row["Feature_4"], row["Feature_5"], row["Feature_6"]]
#         feature_scores = [row["Score_1"], row["Score_2"], row["Score_3"], row["Score_4"], row["Score_5"], row["Score_6"]]
#         # Create the plot
#         plt.figure(figsize=(30, 16))
#         plt.barh(top_features, feature_scores, align="center", alpha=0.7)
#         plt.yticks(fontsize=18)
#         plt.xticks(fontsize=18)
#         plt.xlabel("Feature Importance Score", fontsize=20)
#         plt.ylabel("Feature Name")
#         plt.title(f"Feature Importance Scores for {df_name}", fontsize=20)
#         plt.tight_layout()
#         # Save the plot
#         plt.savefig(os.path.join(plot_dir, f"{df_name}_{row['Model']}_feature_importance.png"))
